In [17]:
!chmod +x env.sh

In [28]:
! ./env.sh

In [9]:
!pip install python-dotenv

In [29]:
from dotenv import load_dotenv

load_dotenv('variables.env', override=True)  # take environment variables from variables.env.

True

In [32]:
%%writefile install-eksctl.sh
# for ARM systems, set ARCH to: `arm64`, `armv6` or `armv7`
ARCH=amd64
PLATFORM=$(uname -s)_$ARCH

curl -sLO "https://github.com/eksctl-io/eksctl/releases/latest/download/eksctl_$PLATFORM.tar.gz"

# (Optional) Verify checksum
curl -sL "https://github.com/eksctl-io/eksctl/releases/latest/download/eksctl_checksums.txt" | grep $PLATFORM | sha256sum --check

tar -xzf eksctl_$PLATFORM.tar.gz -C /tmp && rm eksctl_$PLATFORM.tar.gz

sudo mv /tmp/eksctl /usr/local/bin

Writing install-eksctl.sh


In [33]:
!chmod +x install-eksctl.sh
!./install-eksctl.sh

eksctl_Linux_amd64.tar.gz: OK


In [30]:
%%writefile create-eks-cluster.sh
curl -fsSL https://raw.githubusercontent.com/aws/karpenter-provider-aws/v"${KARPENTER_VERSION}"/website/content/en/preview/getting-started/getting-started-with-karpenter/cloudformation.yaml  > "${TEMPOUT}" \
&& aws cloudformation deploy \
  --stack-name "Karpenter-${CLUSTER_NAME}" \
  --template-file "${TEMPOUT}" \
  --capabilities CAPABILITY_NAMED_IAM \
  --parameter-overrides "ClusterName=${CLUSTER_NAME}"

eksctl create cluster -f - <<EOF
---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig
metadata:
  name: ${CLUSTER_NAME}
  region: ${AWS_DEFAULT_REGION}
  version: "${K8S_VERSION}"
  tags:
    karpenter.sh/discovery: ${CLUSTER_NAME}

iam:
  withOIDC: true
  podIdentityAssociations:
  - namespace: "${KARPENTER_NAMESPACE}"
    serviceAccountName: karpenter
    roleName: ${CLUSTER_NAME}-karpenter
    permissionPolicyARNs:
    - arn:${AWS_PARTITION}:iam::${AWS_ACCOUNT_ID}:policy/KarpenterControllerPolicy-${CLUSTER_NAME}

iamIdentityMappings:
- arn: "arn:${AWS_PARTITION}:iam::${AWS_ACCOUNT_ID}:role/KarpenterNodeRole-${CLUSTER_NAME}"
  username: system:node:{{EC2PrivateDNSName}}
  groups:
  - system:bootstrappers
  - system:nodes
  ## If you intend to run Windows workloads, the kube-proxy group should be specified.
  # For more information, see https://github.com/aws/karpenter/issues/5099.
  # - eks:kube-proxy-windows

managedNodeGroups:
- instanceType: m5.large
  amiFamily: AmazonLinux2
  name: ${CLUSTER_NAME}-ng
  desiredCapacity: 2
  minSize: 1
  maxSize: 10

addons:
- name: eks-pod-identity-agent
EOF

export CLUSTER_ENDPOINT="$(aws eks describe-cluster --name "${CLUSTER_NAME}" --query "cluster.endpoint" --output text)"
export KARPENTER_IAM_ROLE_ARN="arn:${AWS_PARTITION}:iam::${AWS_ACCOUNT_ID}:role/${CLUSTER_NAME}-karpenter"

echo "${CLUSTER_ENDPOINT} ${KARPENTER_IAM_ROLE_ARN}"



Writing create-eks-cluster.sh


In [ ]:
!chmod +x create-eks-cluster.sh
!./create-eks-cluster.sh


Waiting for changeset to be created..

No changes to deploy. Stack Karpenter-kserve-demo is up to date
2024-06-07 06:13:33 [ℹ]  eksctl version 0.181.0
2024-06-07 06:13:33 [ℹ]  using region us-west-2
2024-06-07 06:13:33 [ℹ]  setting availability zones to [us-west-2a us-west-2b us-west-2c]
2024-06-07 06:13:33 [ℹ]  subnets for us-west-2a - public:192.168.0.0/19 private:192.168.96.0/19
2024-06-07 06:13:33 [ℹ]  subnets for us-west-2b - public:192.168.32.0/19 private:192.168.128.0/19
2024-06-07 06:13:33 [ℹ]  subnets for us-west-2c - public:192.168.64.0/19 private:192.168.160.0/19
2024-06-07 06:13:33 [ℹ]  nodegroup "kserve-demo-ng" will use "" [AmazonLinux2/1.29]
2024-06-07 06:13:33 [ℹ]  using Kubernetes version 1.29
2024-06-07 06:13:33 [ℹ]  creating EKS cluster "kserve-demo" in "us-west-2" region with managed nodes
2024-06-07 06:13:33 [ℹ]  1 nodegroup (kserve-demo-ng) was included (based on the include/exclude rules)
2024-06-07 06:13:33 [ℹ]  will create a CloudFormation stack for cluster it

In [38]:
%%bash
CLUSTER_ENDPOINT="$(aws eks describe-cluster --name "${CLUSTER_NAME}" --query "cluster.endpoint" --output text)"
KARPENTER_IAM_ROLE_ARN="arn:${AWS_PARTITION}:iam::${AWS_ACCOUNT_ID}:role/${CLUSTER_NAME}-karpenter"
cat > variables2.env << EOF
CLUSTER_ENDPOINT=${AWS_DEFAULT_REGION}
KARPENTER_IAM_ROLE_ARN=${AWS_ACCESS_KEY_ID}
EOF

In [39]:
load_dotenv('variables2.env', override=True)  # take environment variables from .env.

True

In [40]:
!aws iam create-service-linked-role --aws-service-name spot.amazonaws.com || true
# If the role has already been successfully created, you will see:
# An error occurred (InvalidInput) when calling the CreateServiceLinkedRole operation: Service role name AWSServiceRoleForEC2Spot has been taken in this account, please try a different suffix.


{
    "Role": {
        "Path": "/aws-service-role/spot.amazonaws.com/",
        "RoleName": "AWSServiceRoleForEC2Spot",
        "RoleId": "AROAURO4PORKWYWP7PPEO",
        "Arn": "arn:aws:iam::312384386133:role/aws-service-role/spot.amazonaws.com/AWSServiceRoleForEC2Spot",
        "CreateDate": "2024-06-07T08:08:07+00:00",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Action": [
                        "sts:AssumeRole"
                    ],
                    "Effect": "Allow",
                    "Principal": {
                        "Service": [
                            "spot.amazonaws.com"
                        ]
                    }
                }
            ]
        }
    }
}


In [45]:
%%bash
curl -fsSL -o get_helm.sh https://raw.githubusercontent.com/helm/helm/main/scripts/get-helm-3
chmod 700 get_helm.sh
./get_helm.sh

Verifying checksum... Done.
Preparing to install helm into /usr/local/bin
helm installed into /usr/local/bin/helm


In [46]:
%%bash
# Logout of helm registry to perform an unauthenticated pull against the public ECR
helm registry logout public.ecr.aws

helm upgrade --install karpenter oci://public.ecr.aws/karpenter/karpenter --version "${KARPENTER_VERSION}" --namespace "${KARPENTER_NAMESPACE}" --create-namespace \
  --set "settings.clusterName=${CLUSTER_NAME}" \
  --set "settings.interruptionQueue=${CLUSTER_NAME}" \
  --set controller.resources.requests.cpu=1 \
  --set controller.resources.requests.memory=1Gi \
  --set controller.resources.limits.cpu=1 \
  --set controller.resources.limits.memory=1Gi \
  --wait


Error: not logged in


Release "karpenter" does not exist. Installing it now.


Pulled: public.ecr.aws/karpenter/karpenter:0.36.2
Digest: sha256:1b3b57830f7368b8934651001c1919a284af5a470b751c9d1a4204fed122cb3c


NAME: karpenter
LAST DEPLOYED: Fri Jun  7 08:13:18 2024
NAMESPACE: kube-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [83]:
%%writefile karpenter-cpu-node-pool.yaml
apiVersion: karpenter.sh/v1beta1
kind: NodePool
metadata:
  name: default
spec:
  # Template section that describes how to template out NodeClaim resources that Karpenter will provision
  # Karpenter will consider this template to be the minimum requirements needed to provision a Node using this NodePool
  # It will overlay this NodePool with Pods that need to schedule to further constrain the NodeClaims
  # Karpenter will provision to launch new Nodes for the cluster
  template:
    spec:
      # References the Cloud Provider's NodeClass resource, see your cloud provider specific documentation
      nodeClassRef:
        apiVersion: karpenter.k8s.aws/v1beta1
        kind: EC2NodeClass
        name: default

      # Requirements that constrain the parameters of provisioned nodes.
      # These requirements are combined with pod.spec.topologySpreadConstraints, pod.spec.affinity.nodeAffinity, pod.spec.affinity.podAffinity, and pod.spec.nodeSelector rules.
      # Operators { In, NotIn, Exists, DoesNotExist, Gt, and Lt } are supported.
      # https://kubernetes.io/docs/concepts/scheduling-eviction/assign-pod-node/#operators
      requirements:
        - key: "karpenter.k8s.aws/instance-category"
          operator: In
          values: ["c", "m", "r"]
        - key: "karpenter.k8s.aws/instance-cpu"
          operator: In
          values: ["4", "8", "16", "32"]
        - key: "karpenter.k8s.aws/instance-hypervisor"
          operator: In
          values: ["nitro"]
        - key: "karpenter.k8s.aws/instance-generation"
          operator: Gt
          values: ["2"]
        # - key: "topology.kubernetes.io/zone"
        #   operator: In
        #   values: ["us-west-2a", "us-west-2b"]
        - key: "kubernetes.io/arch"
          operator: In
          values: ["arm64", "amd64"]
        - key: "karpenter.sh/capacity-type"
          operator: In
          values: ["spot", "on-demand"]
        - key: "karpenter.k8s.aws/instance-cpu-manufacturer"
          operator: In
          values: ["intel"]

  # Disruption section which describes the ways in which Karpenter can disrupt and replace Nodes
  # Configuration in this section constrains how aggressive Karpenter can be with performing operations
  # like rolling Nodes due to them hitting their maximum lifetime (expiry) or scaling down nodes to reduce cluster cost
  disruption:
    # Describes which types of Nodes Karpenter should consider for consolidation
    # If using 'WhenUnderutilized', Karpenter will consider all nodes for consolidation and attempt to remove or replace Nodes when it discovers that the Node is underutilized and could be changed to reduce cost
    # If using `WhenEmpty`, Karpenter will only consider nodes for consolidation that contain no workload pods
    consolidationPolicy: WhenEmpty

    # The amount of time Karpenter should wait after discovering a consolidation decision
    # This value can currently only be set when the consolidationPolicy is 'WhenEmpty'
    # You can choose to disable consolidation entirely by setting the string value 'Never' here
    consolidateAfter: 30s

    # The amount of time a Node can live on the cluster before being removed
    # Avoiding long-running Nodes helps to reduce security vulnerabilities as well as to reduce the chance of issues that can plague Nodes with long uptimes such as file fragmentation or memory leaks from system processes
    # You can choose to disable expiration entirely by setting the string value 'Never' here
    expireAfter: 720h

  # Resource limits constrain the total size of the pool.
  # Limits prevent Karpenter from creating new instances once the limit is exceeded.
  limits:
    cpu: "1000"
    memory: 1000Gi

  # Priority given to the NodePool when the scheduler considers which NodePool
  # to select. Higher weights indicate higher priority when comparing NodePools.
  # Specifying no weight is equivalent to specifying a weight of 0.
  weight: 10
---
apiVersion: karpenter.k8s.aws/v1beta1
kind: EC2NodeClass
metadata:
  name: default
spec:
  # Required, resolves a default ami and userdata
  amiFamily: AL2
  role: KarpenterNodeRole-kserve-demo
  # Required, discovers subnets to attach to instances
  # Each term in the array of subnetSelectorTerms is ORed together
  # Within a single term, all conditions are ANDed
  subnetSelectorTerms:
    # Select on any subnet that has the "karpenter.sh/discovery: ${CLUSTER_NAME}"
    # AND the "environment: test" tag OR any subnet with ID "subnet-09fa4a0a8f233a921"
    - tags:
        karpenter.sh/discovery: kserve-demo

  # Required, discovers security groups to attach to instances
  # Each term in the array of securityGroupSelectorTerms is ORed together
  # Within a single term, all conditions are ANDed
  securityGroupSelectorTerms:
    # Select on any security group that has both the "karpenter.sh/discovery: ${CLUSTER_NAME}" tag
    # AND the "environment: test" tag OR any security group with the "my-security-group" name
    # OR any security group with ID "sg-063d7acfb4b06c82c"
    - tags:
        karpenter.sh/discovery: kserve-demo

  blockDeviceMappings:
    - deviceName: /dev/xvda
      ebs:
        volumeSize: 200Gi
        volumeType: gp3



Overwriting karpenter-cpu-node-pool.yaml


In [48]:
%%writefile karpenter-gpu-node-pool.yaml
apiVersion: karpenter.sh/v1beta1
kind: NodePool
metadata:
  name: gpu
spec:
  disruption:
    consolidationPolicy: WhenUnderutilized
  template:
    spec:
      nodeClassRef: 
        apiVersion: karpenter.k8s.aws/v1beta1
        kind: EC2NodeClass
        name: gpu
      requirements:
      - key: "karpenter.k8s.aws/instance-category"
        operator: In
        values: ["g"]
      - key: "karpenter.k8s.aws/instance-gpu-name"
        operator: In
        values: ["a10g"]
      - key: "karpenter.k8s.aws/instance-gpu-count"
        operator: Gt
        values: ["0"]
      - key: "karpenter.k8s.aws/instance-gpu-count"
        operator: Lt
        values: ["10"]
      - key: "karpenter.sh/capacity-type"
        operator: In
        values: ["spot", "on-demand"]
      - key: "kubernetes.io/arch"
        operator: In
        values: ["amd64"]
      - key: "kubernetes.io/os"
        operator: In
        values: ["linux"]
      - key: karpenter.k8s.aws/instance-local-nvme
        operator: Gt
        values: ["100"]
  limits:
    nvidia.com/gpu: "10"
  weight: 1
---
apiVersion: karpenter.k8s.aws/v1beta1
kind: EC2NodeClass
metadata:
  name: gpu
spec:
  # Required, resolves a default ami and userdata
  amiFamily: Ubuntu
  role: KarpenterNodeRole-kserve-demo
  instanceStorePolicy: RAID0
  # Required, discovers subnets to attach to instances
  # Each term in the array of subnetSelectorTerms is ORed together
  # Within a single term, all conditions are ANDed
  subnetSelectorTerms:
    # Select on any subnet that has the "karpenter.sh/discovery: ${CLUSTER_NAME}"
    # AND the "environment: test" tag OR any subnet with ID "subnet-09fa4a0a8f233a921"
    - tags:
        karpenter.sh/discovery: kserve-demo

  # Required, discovers security groups to attach to instances
  # Each term in the array of securityGroupSelectorTerms is ORed together
  # Within a single term, all conditions are ANDed
  securityGroupSelectorTerms:
    # Select on any security group that has both the "karpenter.sh/discovery: ${CLUSTER_NAME}" tag
    # AND the "environment: test" tag OR any security group with the "my-security-group" name
    # OR any security group with ID "sg-063d7acfb4b06c82c"
    - tags:
        karpenter.sh/discovery: kserve-demo

  amiSelectorTerms:
  - name: amazon-eks-gpu-node-1.29-v20240531

  blockDeviceMappings:
    - deviceName: /dev/xvda
      ebs:
        volumeSize: 200Gi
        volumeType: gp3




Writing karpenter-gpu-node-pool.yaml


In [51]:
!curl -LO https://dl.k8s.io/release/v1.29.0/bin/linux/amd64/kubectl
!chmod +x kubectl
!mkdir -p ~/.local/bin
!mv ./kubectl ~/.local/bin/kubectl
%env PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/sagemaker-user/.local/bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1068      0 --:--:-- --:--:-- --:--:--  1069
100 47.4M  100 47.4M    0     0  78.2M      0 --:--:-- --:--:-- --:--:-- 78.2M


In [62]:
!kubectl create -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.15.0/deployments/static/nvidia-device-plugin.yml
!kubectl apply -f karpenter-cpu-node-pool.yaml
# if you are not running in workshop environment, please uncomment the following line
# !kubectl apply -f karpenter-gpu-node-pool.yaml


Error from server (AlreadyExists): error when creating "https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.15.0/deployments/static/nvidia-device-plugin.yml": daemonsets.apps "nvidia-device-plugin-daemonset" already exists
nodepool.karpenter.sh/default created
nodepool.karpenter.sh/gpu created
ec2nodeclass.karpenter.k8s.aws/gpu created


In [58]:
# add tag kubernetes.io/cluster/kserve-demo=owned to ClusterSharedNodeSecurityGroup to create load balancer


env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/sagemaker-user/.local/bin


In [64]:
!sudo apt install jq -y
!sudo apt install cosign -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 9 not upgraded.
Need to get 357 kB of archives.
After this operation, 1087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 1s (641 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package libonig5:amd64.
(Reading database ... 13795 files and directories currently installed.)
Preparing to unpack .../libonig5_6.9.7.1-2build1_amd64.deb ...
7Progress: [  0%] 

In [68]:
!curl -O -L "https://github.com/sigstore/cosign/releases/latest/download/cosign-linux-amd64"
!mv cosign-linux-amd64 ~/.local/bin/cosign
!chmod +x ~/.local/bin/cosign

In [78]:
%env ISTIO_VERSION=1.21.3
!curl -sL https://istio.io/downloadIstioctl | sh -
%env PATH=/home/sagemaker-user/.istioctl/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/sagemaker-user/.local/bin

env: ISTIO_VERSION=1.21.3

istioctl-1.21.3-linux-amd64.tar.gz download complete!

Add the istioctl to your path with:
  export PATH=$HOME/.istioctl/bin:$PATH 

Begin the Istio pre-installation check by running:
	 istioctl x precheck 

Need more information? Visit https://istio.io/docs/reference/commands/istioctl/ 
env: PATH=/home/sagemaker-user/.istioctl/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/sagemaker-user/.local/bin


In [79]:
%%bash
curl -sSL https://github.com/knative/serving/releases/download/knative-v1.13.1/serving-core.yaml \
  | grep 'gcr.io/' | awk '{print $2}' | sort | uniq \
  | xargs -n 1 \
    cosign verify -o text \
      --certificate-identity=signer@knative-releases.iam.gserviceaccount.com \
      --certificate-oidc-issuer=https://accounts.google.com
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.13.1/serving-crds.yaml
kubectl apply -f https://github.com/knative/serving/releases/download/knative-v1.13.1/serving-core.yaml
istioctl install -y
kubectl label namespace knative-serving istio-injection=enabled
# kubectl apply -f knative-serving-peer-auth.yaml
istioctl verify-install
kubectl apply -f https://github.com/knative/net-istio/releases/download/knative-v1.13.1/net-istio.yaml
kubectl apply -f https://github.com/cert-manager/cert-manager/releases/download/v1.14.5/cert-manager.yaml
kubectl apply -f https://github.com/kserve/kserve/releases/download/v0.13.0/kserve.yaml
kubectl apply -f https://github.com/kserve/kserve/releases/download/v0.13.0/kserve-cluster-resources.yaml
#Since you are using Knative 1.8, there is a change that it defaults the domain to svc.cluster.local which is not exposed on ingress, if you want to run the curl command outside of the kube cluster you would need to configure the external domain.
kubectl patch cm config-domain --patch '{"data":{"example.com":""}}' -n knative-serving



- Processing resources for Istio core.
✔ Istio core installed
- Processing resources for Istiod.
- Processing resources for Istiod. Waiting for Deployment/istio-system/istiod
✘ Istiod encountered an error: failed to wait for resource: resources not ready after 5m0s: context deadline exceeded
  
- Processing resources for Ingress gateways.
- Processing resources for Ingress gateways. Waiting for Deployment/istio-system/istio-ingressgat...
✘ Ingress gateways encountered an error: failed to wait for resource: resources not ready after 5m0s: context deadline exceeded
  Deployment/istio-system/istio-ingressgateway (container failed to start: ContainerCreating: )
- Pruning removed resourcesError: failed to install manifests: errors occurred during operation


namespace/knative-serving not labeled
1 Istio control planes detected, checking --revision "default" only
✘ Deployment: istio-ingressgateway.istio-system: Istio installation failed, incomplete or does not match "generated from default": waiting for deployment "istio-ingressgateway" rollout to finish: 0 of 1 updated replicas are available
✘ Deployment: istiod.istio-system: Istio installation failed, incomplete or does not match "generated from default": deployment "istiod" exceeded its progress deadline
✔ Service: istio-ingressgateway.istio-system checked successfully
✔ Service: istiod.istio-system checked successfully
✔ ConfigMap: istio.istio-system checked successfully
✔ ConfigMap: istio-sidecar-injector.istio-system checked successfully
✔ Pod: istio-ingressgateway-59d6b88bc9-mtm6b.istio-system checked successfully
✔ Pod: istiod-865d6658f8-9btsw.istio-system checked successfully
✔ ServiceAccount: istio-ingressgateway-service-account.istio-system checked successfully
✔ ServiceAccount: 

Error: [Istio installation failed, incomplete or does not match "generated from default": waiting for deployment "istio-ingressgateway" rollout to finish: 0 of 1 updated replicas are available, Istio installation failed, incomplete or does not match "generated from default": deployment "istiod" exceeded its progress deadline]


clusterrole.rbac.authorization.k8s.io/knative-serving-istio unchanged
gateway.networking.istio.io/knative-ingress-gateway created
gateway.networking.istio.io/knative-local-gateway created
service/knative-local-gateway created
configmap/config-istio unchanged
peerauthentication.security.istio.io/webhook created
peerauthentication.security.istio.io/net-istio-webhook created
deployment.apps/net-istio-controller unchanged
deployment.apps/net-istio-webhook unchanged
secret/net-istio-webhook-certs unchanged
service/net-istio-webhook unchanged
mutatingwebhookconfiguration.admissionregistration.k8s.io/webhook.istio.networking.internal.knative.dev unchanged
validatingwebhookconfiguration.admissionregistration.k8s.io/config.webhook.istio.networking.internal.knative.dev unchanged
certificate.networking.internal.knative.dev/routing-serving-certs created
namespace/cert-manager unchanged
customresourcedefinition.apiextensions.k8s.io/certificaterequests.cert-manager.io unchanged
customresourcedefinit

Error from server (InternalError): error when creating "https://github.com/kserve/kserve/releases/download/v0.13.0/kserve-cluster-resources.yaml": Internal error occurred: failed calling webhook "clusterservingruntime.kserve-webhook-server.validator": failed to call webhook: Post "https://kserve-webhook-server-service.kserve.svc:443/validate-serving-kserve-io-v1alpha1-clusterservingruntime?timeout=10s": no endpoints available for service "kserve-webhook-server-service"
Error from server (InternalError): error when creating "https://github.com/kserve/kserve/releases/download/v0.13.0/kserve-cluster-resources.yaml": Internal error occurred: failed calling webhook "clusterservingruntime.kserve-webhook-server.validator": failed to call webhook: Post "https://kserve-webhook-server-service.kserve.svc:443/validate-serving-kserve-io-v1alpha1-clusterservingruntime?timeout=10s": no endpoints available for service "kserve-webhook-server-service"
Error from server (InternalError): error when creati

configmap/config-domain patched (no change)


In [80]:
%%bash
# Add kubernetes-dashboard repository
helm repo add kubernetes-dashboard https://kubernetes.github.io/dashboard/
# Deploy a Helm Release named "kubernetes-dashboard" using the kubernetes-dashboard chart
helm upgrade --install kubernetes-dashboard kubernetes-dashboard/kubernetes-dashboard --create-namespace --namespace kubernetes-dashboard

kubectl create serviceaccount dashboard-admin-sa
kubectl create clusterrolebinding dashboard-admin-sa --clusterrole=cluster-admin --serviceaccount=default:dashboard-admin-sa
kubectl apply -f - <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: dashboard-admin-sa-secret
  annotations:
    kubernetes.io/service-account.name: dashboard-admin-sa
type: kubernetes.io/service-account-token
EOF
kubectl describe secret dashboard-admin-sa-secret

"kubernetes-dashboard" has been added to your repositories
Release "kubernetes-dashboard" does not exist. Installing it now.
NAME: kubernetes-dashboard
LAST DEPLOYED: Fri Jun  7 09:06:00 2024
NAMESPACE: kubernetes-dashboard
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
*************************************************************************************************
*** PLEASE BE PATIENT: Kubernetes Dashboard may need a few minutes to get up and become ready ***
*************************************************************************************************

Congratulations! You have just installed Kubernetes Dashboard in your cluster.

To access Dashboard run:
  kubectl -n kubernetes-dashboard port-forward svc/kubernetes-dashboard-kong-proxy 8443:443

NOTE: In case port-forward command does not work, make sure that kong service name is correct.
      Check the services in Kubernetes Dashboard namespace using:
        kubectl -n kubernetes-dashboard get svc

Dashboard will be av

In [ ]:
#kubectl edit svc/kubernetes-dashboard-kong-proxy -n kubernetes-dashboard
#ClusterIP => LoadBalancer and then ctrl+X Y Enter
#Use the CLB addr (https://xxxx) to visit to the dashboard

In [90]:
%%bash
kubectl create namespace kserve-test
# is the following required?
# kubectl label namespace kserve-test istio-injection=enabled
kubectl apply -n kserve-test -f - <<EOF
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "sklearn-iris"
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
      storageUri: "gs://kfserving-examples/models/sklearn/1.0/model"
EOF

inferenceservice.serving.kserve.io "sklearn-iris" deleted
inferenceservice.serving.kserve.io/sklearn-iris created


In [91]:
#wait until READY = TRUE
!kubectl get inferenceservices sklearn-iris -n kserve-test


NAME           URL                                           READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION            AGE
sklearn-iris   http://sklearn-iris.kserve-test.example.com   True           100                              sklearn-iris-predictor-00001   2m18s


In [92]:
%%writefile iris-input.json
{
  "instances": [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}


Writing iris-input.json


In [93]:
%%bash
export INGRESS_HOST=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].port}')
echo ${INGRESS_HOST}:${INGRESS_PORT}
SERVICE_HOSTNAME=$(kubectl get inferenceservice sklearn-iris -n kserve-test -o jsonpath='{.status.url}' | cut -d "/" -f 3)
curl -v -H "Host: ${SERVICE_HOSTNAME}" -H "Content-Type: application/json" "http://${INGRESS_HOST}:${INGRESS_PORT}/v1/models/sklearn-iris:predict" -d @./iris-input.json


a8526a93672a448229e70fde870ffeef-89339606.us-west-2.elb.amazonaws.com:80


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 52.26.94.246:80...
* Connected to a8526a93672a448229e70fde870ffeef-89339606.us-west-2.elb.amazonaws.com (52.26.94.246) port 80 (#0)
> POST /v1/models/sklearn-iris:predict HTTP/1.1
> Host: sklearn-iris.kserve-test.example.com
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 76
> 
} [76 bytes data]
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 21
< content-type: application/json
< date: Fri, 07 Jun 2024 11:08:51 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 8
< 
{ [21 bytes data]
100    97  100    21  100    76   1095   3962 --:--:-- --:--:-- --:--:--  5105
* Connection #0 to host a8526a93672a448229e70fde870ffeef-89339606.us-west-2.elb.amazonaws.com left intact

{"predictions":[1,1]}

In [ ]:
# Download mnist-tensorflow model from gs://kfserving-examples/models/
!aws s3 cp --recursive mnist-tensorflow/ s3://YOUR_BUCKET/mnist-tensorflow/
# Create the service account + IAM Role via eksctl and then modify the service account
!eksctl create iamserviceaccount --name s3-sa --namespace kserve-test --cluster kserve-demo --role-name kserve-demo-s3-access \
    --attach-policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess --approve


In [ ]:
%%writefile s3-sa.yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  name: s3-sa
  annotations:
    eks.amazonaws.com/role-arn: arn:aws:iam::YOUR_ACCOUNT:role/kserve-demo-s3-access # replace with your IAM role ARN
    # serving.kserve.io/s3-endpoint: s3.us-west-2.amazonaws.com # replace with your s3 endpoint e.g minio-service.kubeflow:9000
    # serving.kserve.io/s3-usehttps: "1" # by default 1, if testing with minio you can set to 0
    serving.kserve.io/s3-region: "us-west-2"
    serving.kserve.io/s3-useanoncredential: "false" # omitting this is the same as false, if true will ignore provided credential and use anonymous credentials

In [ ]:
%%writefile mnist-s3.yaml
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "mnist-s3"
spec:
  predictor:
    serviceAccountName: s3-sa
    model:
      modelFormat:
        name: tensorflow
      storageUri: "s3://YOUR_BUCKET/mnist-tensorflow"


In [ ]:
!kubectl apply -f s3-sa.yaml -n kserve-test
!kubectl apply -f mnist-s3.yaml -n kserve-test


In [ ]:
%%bash
SERVICE_HOSTNAME=$(kubectl get inferenceservice mnist-s3 -n kserve-test -o jsonpath='{.status.url}' | cut -d "/" -f 3)

MODEL_NAME=mnist-s3
INPUT_PATH=@./input.json
curl -v -H "Host: ${SERVICE_HOSTNAME}" -H "Content-Type: application/json" "http://${INGRESS_HOST}:${INGRESS_PORT}/v1/models/$MODEL_NAME:predict" -d $INPUT_PATH


In [ ]:
# The following test won't work in the workshop due to no GPU resource availause huggingface cli to download the model weights and then upload s3

In [ ]:
%%writefile huggingface-llama3.yaml
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: llama3
spec:
  predictor:
    serviceAccountName: s3-sa
    scaleTarget: 5
    scaleMetric: concurrency
    # batcher:
    #   maxBatchSize: 4
    #   maxLatency: 500
    model:
      storageUri: "s3://YOUR_BUCKET/Meta-Llama-3-8B"
      modelFormat:
        name: huggingface
      args:
      - --model_name=llama3
      - --dtype=bfloat16
      resources:
        limits:
          cpu: "6"
          memory: 24Gi
          nvidia.com/gpu: "1"
          ephemeral-storage: "40Gi"
        requests:
          cpu: "6"
          memory: 24Gi
          nvidia.com/gpu: "1"
          ephemeral-storage: "20Gi"
